In [7]:
# OPENAI HELPER FUNCTIONS
from ast import Dict, List
from ctypes import Union
import re
from typing import Any, Dict, List, NewType, Optional, Union

from dotenv import load_dotenv
from pathlib import Path
import os
import json
import requests
from tqdm import tqdm

from serpapi import GoogleSearch
import openai

import time

import json

# map src
import sys

sys.path.append("../..")

from src.utils.slack import send_slack_message, URL_MAP

import csv
from datetime import datetime


serp_api_key = os.environ.get("SERP_API_KEY")
openai.api_key = os.environ.get("OPENAI_KEY")

env_path = Path("../..") / ".envprod"
load_dotenv(dotenv_path=env_path)

ISCRAPER_API_KEY = os.environ.get("ISCRAPER_API_KEY")
PROFILE_DETAILS_URL = "https://api.iscraper.io/v2/profile-details"


CURRENT_OPENAI_DAVINCI_MODEL = "text-davinci-003"
CURRENT_OPENAI_CHAT_GPT_MODEL = "gpt-3.5-turbo"
CURRENT_OPENAI_LATEST_GPT_MODEL = "gpt-4"
DEFAULT_SUFFIX = None
DEFAULT_MAX_TOKENS = 16
DEFAULT_TEMPERATURE = 1
DEFAULT_TOP_P = 1
DEFAULT_N = 1
DEFAULT_FREQUENCY_PENALTY = 0
DEFAULT_STOP = None

NUM_GOOGLE_RESULTS_TO_SCRAPE = 10
MAX_NUM_PROFILES_TO_PROCESS = 10


def search_google_news(query, type: Optional[str] = None):
    # https://support.google.com/websearch/answer/2466433?hl=en
    params = {
        "api_key": serp_api_key,
        "engine": "google",
        "q": query,
        "tbm": type,
        "gl": "us",  # US only
        "hl": "en",
        "num": NUM_GOOGLE_RESULTS_TO_SCRAPE,
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    return results


def wrapped_chat_gpt_completion(
    messages: list,
    history: Optional[list] = [],
    max_tokens: Optional[int] = DEFAULT_MAX_TOKENS,
    temperature: Optional[float] = DEFAULT_TEMPERATURE,
    top_p: Optional[float] = DEFAULT_TOP_P,
    n: Optional[int] = DEFAULT_N,
    frequency_penalty: Optional[float] = DEFAULT_FREQUENCY_PENALTY,
):
    """
    Generates a completion using the GPT-3.5-turbo model.

    messages needs to be in the format:
    [
        {
            "role": "user",
            "content": "Hello, how are you?"
        },
        {
            "role": "assistant",
            "content": "I am doing well, how about you?"
        }
        ...
    ]
    """
    if history:
        messages = history + messages

    response = openai.ChatCompletion.create(
        model=CURRENT_OPENAI_LATEST_GPT_MODEL,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        n=n,
        frequency_penalty=frequency_penalty,
    )
    if response is None or response["choices"] is None or len(response["choices"]) == 0:
        return [], ""

    choices = response["choices"]
    top_choice = choices[0]
    preview = top_choice["message"]["content"].strip()

    messages = messages + [{"role": "assistant", "content": preview}]
    return messages, preview


def call_iscraper(linkedin_id: str):
    payload = json.dumps(
        {
            "profile_id": linkedin_id,
            "profile_type": "personal",
            "network_info": True,
        }
    )
    headers = {"X-API-KEY": ISCRAPER_API_KEY, "Content-Type": "application/json"}

    response = requests.request(
        "POST", PROFILE_DETAILS_URL, headers=headers, data=payload
    )
    data = json.loads(response.text)

    return data


def research_corporate_profile_details(company_name: str):
    payload = json.dumps(
        {
            "profile_id": company_name,
            "profile_type": "company",
            "contact_info": True,
            "recommendations": True,
            "related_profiles": True,
            "network_info": True,
        }
    )
    headers = {"X-API-KEY": ISCRAPER_API_KEY, "Content-Type": "application/json"}

    response = requests.request(
        "POST", PROFILE_DETAILS_URL, headers=headers, data=payload
    )

    return json.loads(response.text)


def str_path_to_path_steps(path: str, *delimiters: str):
    steps = re.split("|".join(map(re.escape, delimiters)), path)
    for step in steps:
        if step.isdigit():
            yield int(step)
        else:
            yield step


def deep_get(obj: Union[List, Dict], path: str, default=None) -> Optional[Any]:
    steps = str_path_to_path_steps(path, ".")

    for step in steps:
        if not obj:
            return default

        if isinstance(obj, dict):
            obj = obj.get(step, None)
        elif isinstance(obj, list) and str(step).isnumeric():
            obj = obj[int(step)]
        else:
            try:
                obj = getattr(obj, step)
            except:
                return default

    return obj

In [26]:
def extract_event_company_info(event_query: str):
    # Fetch recent news articles related to the event
    news_results = search_google_news(event_query, "nws")

    # Prepare data for CSV
    csv_data = []

    for article in news_results.get("news_results", []):
        # Prepare the message for Chat GPT to extract the company name
        chat_message = [
            {
                "role": "user",
                "content": f"Which company does this news article pertain to? {article['title']} {article['snippet']}\nOnly respond with the company name. If company not found, return 'none' all lowercase.\nCompany name:",
            }
        ]

        # Extract company name using Chat GPT
        _, company_name = wrapped_chat_gpt_completion(chat_message)

        # If company name is not found, skip the article
        if "none" in company_name.lower():
            continue

        # Append the details to the CSV data list
        csv_data.append(
            {
                "img_url": article.get("thumbnail"),
                "title": article.get("title"),
                "snippet": article.get("snippet"),
                "link": article.get("link"),
                "date": article.get("date"),
                "company_name": company_name,
            }
        )

    # Generate a timestamped CSV file name
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    # file name with timestamp, and the query
    csv_filename = f"{timestamp}-{event_query}.csv"

    # Write the data to a CSV file
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(
            file,
            fieldnames=["img_url", "title", "snippet", "link", "date", "company_name"],
        )
        writer.writeheader()
        writer.writerows(csv_data)

    print(f"Data extraction completed. CSV file created: {csv_filename}")

    return csv_filename


def qualify_company_events(csv_filename: str, ai_qualifying_question: str):
    # Read the existing CSV file
    with open(csv_filename, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        data = list(reader)

    # Process each row to add 'qualified' column
    for row in data:
        # Prepare the message for Chat GPT
        chat_message = [
            {
                "role": "user",
                "content": f"{ai_qualifying_question} Event: {row['title']}. Company: {row['company_name']}.\nOnly respond with 'true' or 'false'.\nResponse:",
            }
        ]

        # Get qualification response using Chat GPT
        _, qualification_response = wrapped_chat_gpt_completion(chat_message)

        # Determine qualification (True/False)
        qualified = "true" in qualification_response.lower()

        # Add 'qualified' field to the row
        row["qualified"] = qualified

    # Write the updated data to a new CSV file
    new_csv_filename = csv_filename.replace(".csv", "-qualified.csv")
    with open(new_csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"Qualification completed. Updated CSV file created: {new_csv_filename}")

    return new_csv_filename


def extract_linkedin_profiles(qualified_csv_filename: str, titles: list):
    # Read the qualified companies
    with open(qualified_csv_filename, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        qualified_companies = [
            row for row in reader if row["qualified"].lower() == "true"
        ]

    # Prepare data for CSV
    profiles_data = []

    # Loop through each company and title
    for company in qualified_companies:
        for title in titles:
            # Construct the Google search query
            query = f'site:linkedin.com/in/ "{company["company_name"]}" "- {title}"'

            # Perform the Google search
            search_results = search_google_news(
                query
            )  # Use your search_google_news function
            organic_results = search_results.get("organic_results", [])

            # Process search results
            for profile in organic_results:
                # Extract relevant profile details
                profile_data = {
                    "img_url": company.get("img_url"),  # Original data
                    "original_title": company.get("title"),  # Original data
                    "snippet": company.get("snippet"),  # Original data
                    "original_link": company.get("link"),  # Original data
                    "date": company.get("date"),  # Original data
                    "company_name": company["company_name"],  # Original data
                    "linkedin_title": title,  # LinkedIn profile title
                    "profile_url": profile.get("link"),  # LinkedIn profile URL
                }
                profiles_data.append(profile_data)

    # Generate a CSV file name
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    csv_filename = f"{timestamp}-linkedin-profiles.csv"

    # Write the data to a CSV file
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=profiles_data[0].keys())
        writer.writeheader()
        writer.writerows(profiles_data)

    print(f"LinkedIn profiles extraction completed. CSV file created: {csv_filename}")

    return csv_filename


def enrich_linkedin_profiles(linkedin_csv_filename: str):
    # Read the LinkedIn profiles CSV
    with open(linkedin_csv_filename, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        linkedin_data = list(reader)

    enriched_data = []

    for row in tqdm(linkedin_data[0:MAX_NUM_PROFILES_TO_PROCESS]):
        # Call the iScraper API
        profile_url = row["profile_url"]
        profile_id = profile_url.split("/in/")[1]
        iscraper_response = call_iscraper(
            profile_id
        )  # Assuming this function is already defined
        time.sleep(1)  # Wait for 1 second between API calls

        # Extract required fields from the iScraper response
        first_name = iscraper_response.get("first_name", "")
        last_name = iscraper_response.get("last_name", "")
        company = deep_get(
            iscraper_response,
            "position_groups.0.profile_positions.0.company",
            default="",
        )
        sub_title = iscraper_response.get("sub_title", "")
        summary = iscraper_response.get("summary", "")
        title = deep_get(
            iscraper_response, "position_groups.0.profile_positions.0.title", default=""
        )
        industry = iscraper_response.get("industry", "")
        profile_picture = iscraper_response.get("profile_picture", "")
        raw_json = json.dumps(iscraper_response)

        # Prepare the enriched row
        enriched_row = {
            **row,
            "prospect_first_name": first_name,
            "prospect_last_name": last_name,
            "prospect_company": company,
            "prospect_sub_title": sub_title,
            "prospect_summary": summary,
            "prospect_title": title,
            "prospect_industry": industry,
            "profile_picture": profile_picture,
            "raw_iscraper_json": raw_json,
        }

        enriched_data.append(enriched_row)

    # Generate a new CSV file name
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    enriched_csv_filename = f"{timestamp}-enriched-linkedin-profiles.csv"

    # Write the data to the new CSV file
    with open(enriched_csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=enriched_data[0].keys())
        writer.writeheader()
        writer.writerows(enriched_data)

    print(
        f"LinkedIn profiles enrichment completed. Enriched CSV file created: {enriched_csv_filename}"
    )

    return enriched_csv_filename


def perform_gpt_checks_on_profiles(enriched_csv_filename: str, roles: list):
    # Read the enriched LinkedIn profiles CSV
    with open(enriched_csv_filename, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        enriched_data = list(reader)

    checked_data = []

    print("Performing GPT checks on LinkedIn profiles...")

    for row in tqdm(enriched_data):
        # Check if the company matches (case-insensitive)
        correct_company = (
            row["company_name"].lower() == row["prospect_company"].lower()
            or row["company_name"].lower() in row["prospect_company"].lower()
            or row["prospect_company"].lower() in row["company_name"].lower()
        )
        row["correct_company"] = correct_company

        # Prepare the message for Chat GPT to check the role
        chat_message = [
            {
                "role": "user",
                "content": f"Does the role '{row['prospect_title']}' match any of these roles: {roles}?\nOnly respond with 'true' or 'false'.\nResponse:",
            }
        ]

        # Get the role match response using Chat GPT
        _, role_match_response = wrapped_chat_gpt_completion(chat_message)

        # Determine if the role matches (True/False)
        correct_role = "true" in role_match_response.lower()
        row["correct_role"] = correct_role

        checked_data.append(row)

    # Generate a new CSV file name
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    checked_csv_filename = f"{timestamp}-checked-linkedin-profiles.csv"

    # Write the data to the new CSV file
    with open(checked_csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=checked_data[0].keys())
        writer.writeheader()
        writer.writerows(checked_data)

    print(
        f"LinkedIn profiles checking completed. Checked CSV file created: {checked_csv_filename}"
    )

    return checked_csv_filename


def create_final_filtered_csv(checked_csv_filename: str):
    # Read the checked LinkedIn profiles CSV
    with open(checked_csv_filename, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        checked_data = list(reader)

    # Filter data where both correct_role and correct_company are True
    filtered_data = [
        row
        for row in checked_data
        if row["correct_role"].lower() == "true"
        and row["correct_company"].lower() == "true"
    ]

    # remove duplicates on linkedin url
    filtered_data = [dict(t) for t in {tuple(d.items()) for d in filtered_data}]

    # Generate a new CSV file name
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    final_csv_filename = f"{timestamp}-final-filtered-profiles.csv"

    # Write the filtered data to the new CSV file
    with open(final_csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=filtered_data[0].keys())
        writer.writeheader()
        writer.writerows(filtered_data)

    print(f"Final filtered CSV file created: {final_csv_filename}")

    return final_csv_filename


def send_slack_notification(
    final_csv_filename: str, trigger_name: str, client_archetype_id: int
):
    # Read data from the final CSV
    with open(final_csv_filename, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        data = list(reader)

    # Prepare sample events for the Slack message
    sample_events = []
    for row in data:
        event = {
            "title": row["original_title"],
            "company": row["company_name"],
            'url': row['original_link'],
            "prospects": [
                {
                    "name": f"{row['prospect_first_name']} {row['prospect_last_name']}",
                    "linkedin_url": row["profile_url"],
                }
            ],
        }

        # if original title already exists, append to prospects
        for sample_event in sample_events:
            if sample_event["title"] == event["title"]:
                sample_event["prospects"].append(event["prospects"][0])
                break
        else:
            sample_events.append(event)

    # Send Slack message
    print("Sending Slack message...")
    result = send_slack_message(
        message="hello",
        webhook_urls=[URL_MAP["eng-sandbox"]],
        blocks=[
            {
                "type": "header",
                "text": {
                    "type": "plain_text",
                    "text": "Trigger ⚡️: " + trigger_name,
                    "emoji": True,
                },
            },
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": f"Identified {len(data)} prospects at {len(set(row['company_name'] for row in data))} companies with recent news",
                },
            },
            {"type": "divider"},
            *[
                {
                    "type": "section",
                    "text": {
                        "type": "mrkdwn",
                        "text": f"- *{event['company'].replace('.', ' ')}: _<{event['url']}|'{event['title'][0:50] + ('...' if len(event['title']) > 50 else '')}'>_*:\n\t\t- {len(event['prospects'])} prospects like <{event['prospects'][0]['linkedin_url']}|{event['prospects'][0]['name']}> were identified",
                    },
                }
                for event in sample_events
            ],
            # button to go to campaign
            {
                "type": "actions",
                "elements": [
                    {
                        "type": "button",
                        "text": {"type": "plain_text", "text": "View Prospects"},
                        "url": f"https://app.sellscale.com/contacts?campaign_id={client_archetype_id}",
                    }
                ],
            },
        ],
    )


def generate_upload_to_campaign_csv(final_csv_filename: str):
    # Read data from the final CSV
    with open(final_csv_filename, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        data = list(reader)

    # Prepare data for the upload_to_campaign CSV
    campaign_data = []
    for row in data:
        campaign_row = {
            "first_name": row["prospect_first_name"],
            "last_name": row["prospect_last_name"],
            "linkedin_url": row["profile_url"],
            "company": row["company_name"],
            "custom_data": f"{row['company_name']} was recently in the news titled '{row['original_title']}'",
        }
        campaign_data.append(campaign_row)

    # Generate a new CSV file name
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    campaign_csv_filename = f"{timestamp}-upload-to-campaign.csv"

    # Write the campaign data to the new CSV file
    with open(campaign_csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=campaign_data[0].keys())
        writer.writeheader()
        writer.writerows(campaign_data)

    print(f"Upload to Campaign CSV file created: {campaign_csv_filename}")

    return campaign_csv_filename


# INPUTS
trigger_name = "Recent Data Leak Companies - DevOps / Security Engineers"
news_event_query = "data leak"
ai_company_qualifying_question = "the company needs to be a tech company and not a non-profit or government organization"
linkedin_titles = ["devops engineer", "site reliability engineer", "security engineer"]
client_archetype_id = 498

# OUTPUTS
# Example usage
print(
    "\n\n############################\n📰 Extracting company news events...\n############################")
raw_company_events_filename = extract_event_company_info(news_event_query)
qualified_csv_filename = qualify_company_events(
    raw_company_events_filename, ai_company_qualifying_question
)

print(
    "\n\n############################\n🧞‍♂️ Extracting LinkedIn profiles from events...\n############################")
linkedin_profiles_csv = extract_linkedin_profiles(
    qualified_csv_filename, linkedin_titles
)
enriched_linkedin_profiles_csv = enrich_linkedin_profiles(linkedin_profiles_csv)

print(
    "\n\n############################\n🤖 Performing GPT checks on LinkedIn profiles...\n############################")
checked_linkedin_profiles_csv = perform_gpt_checks_on_profiles(
    enriched_linkedin_profiles_csv, linkedin_titles
)

print(
    "\n\n############################\n🗄 Creating final filtered CSV...\n############################")
final_csv_filename = create_final_filtered_csv(checked_linkedin_profiles_csv)

print(
    "\n\n############################\n💬 Sending Slack notification...\n############################")
send_slack_notification(final_csv_filename, trigger_name, client_archetype_id)

print("\n\n############################\n📤 Generating Upload to Campaign CSV...\n############################")
upload_to_campaign_csv = generate_upload_to_campaign_csv(final_csv_filename)
print("Step 1: Upload CSV to the SellScale Campaign " + "https://app.sellscale.com/contacts?campaign_id=" + str(client_archetype_id))
print("Step 2: Go to Settings > Advanced > Custom Data Point and upload the CSV file there")

print(
    "\n\n############################\n✅ Trigger Run Done!\n############################"
)



############################
💬 Sending Slack notification...
############################
Sending Slack message...


############################
📤 Generating Upload to Campaign CSV...
############################
Upload to Campaign CSV file created: 20231120-141948-upload-to-campaign.csv
Step 1: Upload CSV to the SellScale Campaign https://app.sellscale.com/contacts?campaign_id=498
Step 2: Go to Settings > Advanced > Custom Data Point and upload the CSV file there


############################
✅ Trigger Run Done!
############################
